In [5]:
# This step loads the input dataset and creates two lists : udataList and movies

import csv

udata = []
with open('ml-100k/u.data', 'r') as f:
    reader1 = csv.reader(f)
    udata = list(reader1)

udataList = []
movies = []
for i in udata:
    udataList.append(i[0].split('\t'))
    if i[0].split('\t')[1] not in movies:
        movies.append(i[0].split('\t')[1])

In [16]:
# This step calculates the RMSE and MAE over a 5-fold cross validation. The idea is to select 100 random movies and put
# 10 records of each of those random movies in training and remaining in test dataset. 

import numpy as np
from surprise import Dataset, Reader
from surprise import SVD, accuracy
import os

diction = {}

for i in range(0,5):
    reader = Reader(line_format='user item rating', sep='\t')
    
    # Select 100 random movies from the list of all movies
    random100Movies = np.random.choice(movies, 100, replace=False)
    for j in random100Movies:
        diction.update({j:0})
    
    # Create training and test datasets
    with open('test','w') as test, open('train','w') as train:
        for i in udataList:
            if i[1] not in random100Movies:
                train.write(str(i[0])+"\t"+str(i[1])+"\t"+str(i[2])+"\n")
            elif i[1] in random100Movies and diction[i[1]] <= 1:
                train.write(str(i[0])+"\t"+str(i[1])+"\t"+str(i[2])+"\n")
                diction[i[1]] += 1
            else:
                test.write(str(i[0])+","+str(i[1])+","+str(i[2])+"\n")
    
    file_path = os.path.expanduser('train')
    traindata = Dataset.load_from_file(file_path, reader=reader)
    
    # Convert testset in the required format
    with open('test', 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
    li = []
    for line in range(1, len(data)):
        li.append(tuple(data[line]))
    testset = []
    for i in li:
        testset.append(tuple([i[0],i[1],int(i[2])]))
    
    # Calculate RMSE and MAE
    algo = SVD()
    trainset = traindata.build_full_trainset()
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)
    accuracy.mae(predictions, verbose=True)


RMSE: 1.0545
MAE:  0.8517
RMSE: 1.0572
MAE:  0.8668
RMSE: 1.0892
MAE:  0.8961
RMSE: 1.0686
MAE:  0.8782
RMSE: 1.0204
MAE:  0.8266
